In [1]:
import numpy as np

# 家電

## 定義

省エネ基準の解説書では以下のように定義されている。

>一般消費者の生活の用に供され、電気を機能上重要な作動のために使用する機械器具及びこれらの電源として使用される電池類で、商用交流電源を使用するもののほか、電池を使用するものを含む。ただし、住宅設備としてあらかじめ工事により住宅に設置されることを目的として設計、製造されたものは除く。

https://www.kenken.go.jp/becc/documents/house/1_181016_v06_PVer0205.pdf

p.1-7　2.55節

つまり、工事の発生しない「電気機械器具」および「電池」を対象とする？

## 年間消費電力量の計算

### 家電群の年間消費電力量

ここでは、住宅に持ち込まれた評価対象となる全ての家電を家電群と呼称する。  
家電群の年間消費電力量$E_{elc,HEAs,y}$ [Wh/year] は、住宅に持ち込まれた$i$個の家電の年間消費電力$E_{elc,HEA,i,y}$ [Wh/year]の積算値である。

$$
E_{elc,HEAs,y} = \sum E_{elc,HEA,i,y}
$$

In [2]:
def get_E_elc_HEAs(E_elc_HEAs):
    return np.sum(E_elc_HEAs)

### 家電毎の年間消費電力

家電$i$の年間消費電力量$E_{elc,HEAs,i,y}$は、計算ステップ$step$時の家電$i$の消費電力量$E_{elc,HEA,i,step}$ [Wh] の積算値である。  
家電の立ち上がりや経過による変動に対応するため、計算ステップの総和は通年とするがその間隔は家電毎に任意とする。  
他の住宅設備との電力収支を見る場合には、計算ステップ間隔は揃えた方が望ましい。  

$$
E_{elc,HEA,i,y} = \frac{1}{\eta_{batt-chrg,HEA,i} \times \eta_{batt-dischrg,HEA,i}} \times \sum \frac{t_{sec,step} \times P_{elc,HEA,i,step}}{3600}　 
$$

$\eta_{batt-chrg,HEA,i}$ :家電$i$に内蔵された充電池(<u>batt</u>ery)の充電(<u>ch</u>a<u>rg</u>e)効率（充電池を内蔵しない場合は1とする） [-]  
$\eta_{batt-dischrg,HEA,i}$ :家電$i$に内蔵された充電池の放電(<u>disch</u>a<u>rg</u>e)効率（充電池を内蔵しない場合は1とする） [-]  
$t_{sec,step}$ :家電$i$の計算ステップ$step$の秒数 [s]  
$P_{elc,HEA,i,step}$ :家電$i$の計算ステップ$step$時の消費電力(<u>el</u>e<u>c</u>tric <u>P</u>ower) [W]  


計算ステップ$step$時の家電$i$の消費電力$P_{elc,HEA,i,step}$ は、在室者や生活スケジュールから設定される家電の稼働状態のスケジュールと家電毎の各稼働状態の消費電力から求まる。

In [3]:
def get_E_elc_HEA(eta_batt_chrg, eta_batt_dischrg, t_sec_steps, P_elc_HEA_steps):
    E_elc_HEA = 1 / (eta_batt_chrg * eta_batt_dischrg) * np.sum(t_sec_steps * P_elc_HEA_steps)
    return E_elc_HEA

### 家電iの稼働状態スケジュールの書式と作成する計算パラメーター用辞書データ

在室者や生活スケジュールから設定される家電の稼働スケジュールは、以下の書式とする。  
これは、スケジュールプログラムで作成するため、家電とは独立している。


|計算ステップ(step)  |0  |1  |…　|n  |  
|:-----------:| :-:| :-:| :-:| :-:|  
|稼働開始時間(t_step_strt) | 1/1 0:00 | 1/1 0:15 | … | 12/31 23:45 | 
|稼働状態(Stat_num) | 3 | 1 | … | 2 |   

上記の書式から以下の辞書を作成する
* (key)計算ステップ番号
* 稼働開始時間
* 稼働状態値
* 継続秒数

In [4]:
stp = np.array([0,1,2])
tm = np.array(['2001-01-01T00:00:00', '2001-01-01T00:15:00', '2001-12-31T23:45:00'], dtype='datetime64')
stat = np.array([3,1,2])
sec = np.array([(np.datetime64(tm[1]) - np.datetime64(tm[0])).item().total_seconds(), \
                (np.datetime64(tm[2]) - np.datetime64(tm[1])).item().total_seconds(), \
                (np.datetime64('2002-01-01T00:00:00') - np.datetime64(tm[2])).item().total_seconds()])
schedule_of_HEA = dict(zip(stp, zip(tm, stat, sec)))

print(schedule_of_HEA)

{0: (numpy.datetime64('2001-01-01T00:00:00'), 3, 900.0), 1: (numpy.datetime64('2001-01-01T00:15:00'), 1, 31534200.0), 2: (numpy.datetime64('2001-12-31T23:45:00'), 2, 900.0)}


### 家電iに必要な関数

家電$i$に必要な関数は、スケジュールから渡される稼働状態値の消費電力を返す。  
これは各家電毎に定義する。  

例えば、  
ドライヤーは時刻変動による消費電力の変化が無いため、計算ステップ時の稼働状態（ON/OFF）のみを引数として消費電力を返すことが出来る。  


一方で  
冷蔵庫など周囲空気の影響を受ける場合は「時刻」「気象条件」などの複数の引数が必要になってしまう。  

このような場合は、辞書データを増やすのではなく
家電関数を呼び出した際に、家電側の初期化関数で「気象条件など」読み込みを行う形にしたい。  

ただし、  
気象データ等のデータベースは初回のみ読み込みをしたい。さらに、家電の計算を終了する際に破棄するコードが余分に必要になる。  



$$
P_{elc,HEA,i,step} =
\begin{cases}
P_{elc,state,0} & 稼働状態1\\
P_{elc,state,1} & 稼働状態2\\
… & …\\
P_{elc,state,n} & 稼働状態n\\
\end{cases}
$$

$P_{elc,state,n}$ :稼働状態$n$の消費電力 [W] 


| 稼働条件 | 消費電力 | 説明 |   
|:-------:|:-------:|:----:|    
|0        |0        |停止 |  
|1        |3      |休止 |
|2        |20      |待機 |
|3        |100      |低負荷時 |
|…        |…      |… |
|n        |300      |高負荷時 |


In [5]:
def get_E_elc_HEA_sample_steps(dictionary_of_HEA_schedule):
    
    key_list = dictionary_of_HEA_schedule.keys()
    t_sec_steps = np.zeros(len(key_list))
    P_elc_HEA_steps = np.zeros(len(key_list))
    
    for i in range(len(key_list)):
        t_sec_steps[i] = dictionary_of_HEA_schedule[i][2]
        if dictionary_of_HEA_schedule[i][1] == 0:
            P_elc_HEA_steps[i] = 0 
        elif dictionary_of_HEA_schedule[i][1] == 1:
            P_elc_HEA_steps[i] = 3
        elif dictionary_of_HEA_schedule[i][1] == 2:
            P_elc_HEA_steps[i] = 20 
        elif dictionary_of_HEA_schedule[i][1] == 3:
            P_elc_HEA_steps[i] = 100 
        else:
            raise ValueError(item)
    
    return t_sec_steps, P_elc_HEA_steps

In [6]:
get_E_elc_HEA_sample_steps(schedule_of_HEA)

(array([  9.00000000e+02,   3.15342000e+07,   9.00000000e+02]),
 array([ 100.,    3.,   20.]))

## サンプル計算

### 計算条件

- 家電1：
    - 内蔵充電池：なし
    - スケジュールは　sch1　
    - 稼働状態は　get_E_elc_HEA_sample_steps　
- 家電2：
    - 内蔵充電池：あり（充電効率　0.95、放電効率 0.96）
    - スケジュールは　sch2　
    - 稼働状態は　get_E_elc_HEA_sample_steps

In [7]:
#最終的にはスケジュールプログラムからsh1を直接読み込む
stp1 = np.array([0,1,2])
tm1 = np.array(['2001-01-01T00:00:00', '2001-01-01T00:30:00', '2001-12-29T23:45:00'], dtype='datetime64')
stat1 = np.array([3,1,2])
sec1 = np.array([(np.datetime64(tm1[1]) - np.datetime64(tm1[0])).item().total_seconds(), \
                (np.datetime64(tm1[2]) - np.datetime64(tm1[1])).item().total_seconds(), \
                (np.datetime64('2002-01-01T00:00:00') - np.datetime64(tm1[2])).item().total_seconds()])
sch1 = dict(zip(stp1, zip(tm1, stat1, sec1)))

In [8]:
#最終的にはスケジュールプログラムからsh2を直接読み込む
stp2 = np.array([0,1,2])
tm2 = np.array(['2001-01-01T00:00:00', '2001-01-01T00:15:00', '2001-12-31T05:45:00'], dtype='datetime64')
stat2 = np.array([3,1,2])
sec2 = np.array([(np.datetime64(tm2[1]) - np.datetime64(tm2[0])).item().total_seconds(), \
                (np.datetime64(tm2[2]) - np.datetime64(tm2[1])).item().total_seconds(), \
                (np.datetime64('2002-01-01T00:00:00') - np.datetime64(tm2[2])).item().total_seconds()])
sch2 = dict(zip(stp2, zip(tm2, stat2, sec2)))

In [9]:
def sample_calc():
    a1, b1 =get_E_elc_HEA_sample_steps(sch1)
    a2, b2 =get_E_elc_HEA_sample_steps(sch2)
    
    d = np.array([get_E_elc_HEA(1, 1, a1, b1), get_E_elc_HEA(0.95, 0.96, a2, b2)])
    return get_E_elc_HEAs(d)

In [10]:
sample_calc()

202792736.84210527